In [95]:
import numpy as np

In [96]:
#defining the state
environment_rows = 12
environment_columns = 12

#defining and initializing the qtable to all zeros
#6 represents the number of actions we have
# q_values = np.zeros((environment_rows, environment_columns, 4))

#define actions
actions = ['up', 'right', 'down', 'left']

In [97]:
def init_reward_mat(environment_rows, environment_columns, row_pass_reward, column_pass_reward):
  #defining the rewards 
  rewards = np.full((environment_rows, environment_columns), -1)

  #drop off locations
  # rewards[0,0] = -1 
  rewards[row_pass_reward,column_pass_reward] = 10 #passenger starts here
  # rewards[environment_rows-2,0] = -1 #drop passenger here
  # rewards[environment_rows-2,environment_columns-2] = -1 #wrong drop off location

  #defining the outer walls
  rewards[:,0] = -100
  rewards[0,:] = -100
  rewards[11,:] = -100
  rewards[:,11] = -100

  #defining the inner walls
  rewards[1:5,3] = -100
  rewards[1:5,8] = -100
  rewards[7:11,5] = -100
  rewards[7:11,6] = -100

  q_values = np.zeros((environment_rows, environment_columns, 4))

  return rewards, q_values

In [98]:
rewards, q_values = init_reward_mat(environment_rows, environment_columns, 1, 10)
# rewards_drop, q_values_2 = init_reward_mat(environment_rows, environment_columns, 10, 1)
#print the rewards matrix
for row in rewards:
  print(row)

[-100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100]
[-100   -1   -1 -100   -1   -1   -1   -1 -100   -1   10 -100]
[-100   -1   -1 -100   -1   -1   -1   -1 -100   -1   -1 -100]
[-100   -1   -1 -100   -1   -1   -1   -1 -100   -1   -1 -100]
[-100   -1   -1 -100   -1   -1   -1   -1 -100   -1   -1 -100]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1 -100]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1 -100]
[-100   -1   -1   -1   -1 -100 -100   -1   -1   -1   -1 -100]
[-100   -1   -1   -1   -1 -100 -100   -1   -1   -1   -1 -100]
[-100   -1   -1   -1   -1 -100 -100   -1   -1   -1   -1 -100]
[-100   -1   -1   -1   -1 -100 -100   -1   -1   -1   -1 -100]
[-100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100]


In [99]:
#check if the taxi and the pass are at the same location
def terminal_state(current_row_index_taxi, current_column_index_taxi):
  if rewards[current_row_index_taxi][current_column_index_taxi] == -1:
    return False
  return True

#define a random starting location for the taxi
def get_starting_location_taxi():
  current_row_index_taxi = np.random.randint(environment_rows-1)
  current_column_index_taxi = np.random.randint(environment_columns-1)
  # print(current_row_index_taxi)
  # print(current_column_index_taxi)

  while terminal_state(current_row_index_taxi, current_column_index_taxi):
    current_row_index_taxi = np.random.randint(environment_rows-1)
    current_column_index_taxi = np.random.randint(environment_rows-1)
  return current_row_index_taxi, current_column_index_taxi

def get_next_action(current_row_index_taxi, current_column_index_taxi, epsilon,q_values):
  if np.random.random() < epsilon: # this is for the exploitation part
    return np.argmax(q_values[current_row_index_taxi, current_column_index_taxi])
  else: #choose a random action -> this is for the exploration part
    return np.random.randint(4)

def get_next_location(current_row_index_taxi, current_column_index_taxi, action_index):
  new_row_index = current_row_index_taxi
  new_column_index = current_column_index_taxi
  if ((actions[action_index] == 'up') & (current_row_index_taxi > 0)):
    new_row_index = new_row_index - 1
  elif actions[action_index] == 'right' and current_column_index_taxi < environment_columns - 1:
    new_column_index = new_column_index + 1
  elif actions[action_index] == 'down' and current_row_index_taxi < environment_rows - 1:
    new_row_index = new_row_index + 1
  elif actions[action_index] == 'left' and current_row_index_taxi > 0:
    new_column_index = new_column_index - 1
  #maybe we need to add here the option of drop off and pickup
  return new_row_index, new_column_index

#getting the shortest path between the taxi and pass
def get_shortest_path(start_row_index_taxi, start_column_index_taxi):
  if terminal_state(start_row_index_taxi, start_column_index_taxi):
    return []
  else: #if they are not at the same location
    current_row_index, current_column_index = start_row_index_taxi, start_column_index_taxi
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    while not terminal_state(current_row_index, current_column_index):
      action_index = get_next_action(current_row_index, current_column_index, 1, q_values)
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      # print(current_row_index, current_column_index)
      shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [100]:
#defining parameters for the runs 
epsilon = 0.9
discount_factor = 0.9
learning_rate = 0.9

In [101]:
def update_values():
  for episode in range(2000):
    row_index,column_index = get_starting_location_taxi()
    while not terminal_state(row_index, column_index):
      action_index = get_next_action(row_index, column_index, epsilon,q_values)

      old_row_index, old_column_index = row_index, column_index
      row_index, column_index = get_next_location(row_index, column_index, action_index)

      reward = rewards[row_index, column_index]
      old_q_value = q_values[old_row_index, old_column_index, action_index]
      temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

      new_q_value = old_q_value + (learning_rate * temporal_difference)
      q_values[old_row_index, old_column_index, action_index] = new_q_value

  print("Complete")

In [102]:
#calling the function for the first run (taxi to passenger)
update_values()

Complete


In [103]:
shortest = get_shortest_path(1,1)
print(shortest)

#after getting the path from the taxi to the passnger, we'll define new rewards and q_values to find the shortest
#path from the taxi with the passnger to paasengers drop-off location. Meaning we're initializing everything again. 
rewards, q_values = init_reward_mat(environment_rows, environment_columns, 10, 1)
#calling again the function update_values for the 2nd run, taxi with passenger to drop-off location of passenger 
update_values()

[[1, 1], [2, 1], [3, 1], [4, 1], [4, 2], [5, 2], [6, 2], [6, 3], [6, 4], [6, 5], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [4, 9], [3, 9], [2, 9], [2, 10], [1, 10]]
Complete


In [104]:
shortest_2 = get_shortest_path(1,10)
shortest_print = []
for elem in shortest[:-1]:
  shortest_print.append(elem)
for elem in shortest_2:
  shortest_print.append(elem)

print(shortest_print)

[[1, 1], [2, 1], [3, 1], [4, 1], [4, 2], [5, 2], [6, 2], [6, 3], [6, 4], [6, 5], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [4, 9], [3, 9], [2, 9], [2, 10], [1, 10], [2, 10], [3, 10], [3, 9], [4, 9], [5, 9], [5, 8], [5, 7], [5, 6], [5, 5], [6, 5], [6, 4], [6, 3], [7, 3], [7, 2], [7, 1], [8, 1], [9, 1], [10, 1]]
